# Metric Hacking knowing the future

I analyzed the metrics given in competition and open solutions that score 17.396+. During the process, I noticed a strange and common tendency among all such solutions. Firstly, I will analyze some of the top solutions, and then I will present my solution, which took 9th once (4 days ago).

Reference:
- [Competition](https://www.kaggle.com/competitions/hull-tactical-market-prediction)
- Supplementary dataset: [S&P Historical Data for Hull Tactical Competition](https://www.kaggle.com/datasets/ambrosm/s-and-p-historical-data-for-hull-tactical-competition)
- [Solution 1 (hull-tactical-ensemble-of-solutions)](https://www.kaggle.com/code/nina2025/hull-tactical-ensemble-of-solutions)
- [Solution 2 (hull-starter-notebook)](https://www.kaggle.com/code/mafiosoquasar/hull-starter-notebook)
- [Solution 3 (hull-tactical-market-prediction-optimization)](https://www.kaggle.com/code/shreyashpatil217/hull-tactical-market-prediction-optimization)

In [ ]:
import matplotlib.pyplot as plt
import os
from warnings import filterwarnings
filterwarnings("ignore")
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import kaggle_evaluation.default_inference_server
from tqdm.auto import tqdm
from scipy.optimize import minimize, Bounds
import polars as pl

## Sharpe metric

I rewrote the metric to make it easier to understand. The changed places contain the commented source code. You can see for yourself by opening brackets/subtracting/multiplying/etc. or replacing with original metric that this metric was not actually changed.

Ideas:
1) a known optimal $strategy\_returns$ allows us to get an optimal $prediction = \frac{strategy\_returns - risk\_free\_rate}{forward\_returns - risk\_free\_rate}$
2) we need to minimize $strategy\_std$ in some way

In [ ]:
MIN_INVESTMENT = 0
MAX_INVESTMENT = 2

class ParticipantVisibleError(Exception):
    pass

def get_score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    solution = solution.copy()
    solution['position'] = submission['prediction']

    if solution['position'].max() > MAX_INVESTMENT:
        raise ParticipantVisibleError(f'Position of {solution["position"].max()} exceeds maximum of {MAX_INVESTMENT}')
    if solution['position'].min() < MIN_INVESTMENT:
        raise ParticipantVisibleError(f'Position of {solution["position"].min()} below minimum of {MIN_INVESTMENT}')

    solution['strategy_returns'] = solution['risk_free_rate'] + solution['position'] * (solution['forward_returns'] - solution['risk_free_rate']) #solution['risk_free_rate'] * (1 - solution['position']) + solution['position'] * solution['forward_returns']

    strategy_excess_returns = solution['position'] * (solution['forward_returns'] - solution['risk_free_rate']) # solution['strategy_returns'] - solution['risk_free_rate']
    strategy_excess_cumulative = (1 + strategy_excess_returns).prod()
    strategy_mean_excess_return = (strategy_excess_cumulative) ** (1 / len(solution)) - 1
    strategy_std = solution['strategy_returns'].std()
    # strategy_volatility = float(strategy_std * np.sqrt(trading_days_per_yr) * 100)

    trading_days_per_yr = 252
    if strategy_std == 0:
        raise ZeroDivisionError
    sharpe = strategy_mean_excess_return / strategy_std * np.sqrt(trading_days_per_yr)

    market_excess_returns = solution['forward_returns'] - solution['risk_free_rate']
    market_excess_cumulative = (1 + market_excess_returns).prod()
    market_mean_excess_return = (market_excess_cumulative) ** (1 / len(solution)) - 1
    market_std = solution['forward_returns'].std()
    # market_volatility = float(market_std * np.sqrt(trading_days_per_yr) * 100)
    # if market_volatility == 0:
    #    raise ParticipantVisibleError('Division by zero, market std is zero')

    excess_vol = max(0, strategy_std / market_std - 1.2) if market_std > 0 else 0 # excess_vol = max(0, strategy_volatility / market_volatility - 1.2) if market_volatility > 0 else 0
    vol_penalty = 1 + excess_vol

    return_gap = max(0, (market_mean_excess_return - strategy_mean_excess_return) * 100 * trading_days_per_yr)
    return_penalty = 1 + (return_gap**2) / 100

    adjusted_sharpe = sharpe / (vol_penalty * return_penalty)
    return min(float(adjusted_sharpe), 1_000_000)

## Analyzing Solutions

I ran all three solutions and made predictions for a public leaderboard (last 180 rows)

In [ ]:
solution1 = [0.09850032923605051, 0.05235900080485686, 7.145334589634184e-07, 5.401765621431342e-09, 5.401765621431342e-09, 5.401765621431342e-09, 5.401765621431342e-09, 0.046487749137963684, 0.10261985680430158, 5.401765621431342e-09, 0.44325322344178797, 5.401765621431342e-09, 0.41444719283572073, 0.47415326517510237, 0.03180970139014769, 1.2392809638126418e-08, 0.058136365136437114, 0.06387797301314817, 0.10521257270931035, 0.10833969337721673, 5.401765621431342e-09, 5.401765621431342e-09, 0.06815463330777216, 5.401765621431342e-09, 0.11041815264741624, 5.401765621431342e-09, 5.401765621431342e-09, 0.08777125075755203, 0.14747016903721633, 0.1733137091190776, 5.401765621431342e-09, 0.08646374057391881, 0.9661971062684696, 5.401765621431342e-09, 0.05512143284877802, 1.4502532536920743e-07, 0.20706086000345836, 0.26204707452348835, 5.401765621431342e-09, 5.401765621431342e-09, 5.401765621431342e-09, 5.401765621431342e-09, 1.6951991252584104, 5.401765621431342e-09, 0.037179824461836646, 5.401765621431342e-09, 5.401765621431342e-09, 0.05424761765071554, 5.401765621431342e-09, 0.10553284528689265, 5.401765621431342e-09, 5.401765621431342e-09, 0.11167332657495752, 5.401765621431342e-09, 0.028029015653986696, 0.07611371819852936, 5.401765621431342e-09, 0.05352373982679573, 5.401765621431342e-09, 1.9999964044674685, 0.03237542619077603, 0.2567228311296155, 5.401765621431342e-09, 5.401765621431342e-09, 5.401765621431342e-09, 0.0877229570590081, 0.21597332301703642, 0.09312527735118725, 5.401765621431342e-09, 5.401765621431342e-09, 2.260844691142288e-08, 5.401765621431342e-09, 0.014192128365723275, 5.401765621431342e-09, 0.03246928758941478, 0.06012959671140469, 5.401765621431342e-09, 5.401765621431342e-09, 0.4552152048109276, 5.401765621431342e-09, 0.022182456967572595, 0.03740119260864038, 0.027464106763858034, 0.0813065785936635, 1.9999964082948227, 0.09360530242858484, 1.9999964094796345, 0.08296825157547842, 0.03913364183716176, 5.401765621431342e-09, 5.401765621431342e-09, 0.14198769099038985, 0.08428771770720324, 5.827377078329442e-08, 0.017434124359492407, 0.08868841606625301, 0.5148224883645147, 0.12099818533887309, 0.09269892697315585, 0.6182301925065126, 5.401765621431342e-09, 5.401765621431342e-09, 1.9999964091434315, 5.401765621431342e-09, 0.027766703680429702, 5.401765621431342e-09, 0.15154576380911697, 6.925738123090853e-08, 0.10476246381489435, 0.10344321319366406, 1.2949486967862423e-07, 5.401765621431342e-09, 0.056680629100654675, 0.7807093031070045, 0.10401089780963488, 5.401765621431342e-09, 0.15025181856566086, 5.401765621431342e-09, 0.06122558547770289, 5.401765621431342e-09, 1.3787133907934599e-07, 5.401765621431342e-09, 0.0591079191562411, 0.05288140559520231, 1.4566896595304644, 0.07499502049676807, 0.11955094301468444, 0.12436937724413805, 1.2566200500127187e-07, 0.13120770262394302, 0.07426871516774425, 5.401765621431342e-09, 1.0965138573122475e-07, 0.09811436176240641, 0.2157081886421674, 5.401765621431342e-09, 0.3289068662332828, 5.401765621431342e-09, 0.18030388044367945, 0.09623825562991818, 9.659410033366654e-08, 0.3313043991167185, 1.0180892354126845e-06, 0.06866179064724014, 1.9999964043971037, 0.14103187183257637, 1.306453779697317e-07, 5.401765621431342e-09, 5.9302126182668496e-08, 5.401765621431342e-09, 5.401765621431342e-09, 0.038215308898954956, 5.401765621431342e-09, 0.07682296020302597, 8.94547106795783e-08, 0.07549194665308165, 8.726947044818633e-09, 0.05495932701720875, 0.17690364098034606, 7.167089219830786e-07, 5.401765621431342e-09, 1.3339912738284889e-07, 5.401765621431342e-09, 5.401765621431342e-09, 5.401765621431342e-09, 0.03799538024966466, 5.401765621431342e-09, 0.14362784434750195, 0.27323104227031153, 0.17138737083238648, 5.401765621431342e-09, 5.401765621431342e-09, 0.11036205518499337, 0.07087325221189124, 5.401765621431342e-09, 0.2543178017593521, 0.2711432477935047, 0.21357160214132562, 0.07168115153531147, 1.0157978438774137e-06]
solution2 = [0.09862743886370412, 0.052498003185370255, 0.00011206284402413593, 7.237538034328989e-09, 7.237618633278696e-09, 7.237661973108939e-09, 7.23789685718028e-09, 0.04662826485469454, 0.10274590477399317, 7.237643413898407e-09, 0.44328552367784807, 7.237766808543503e-09, 0.41448726640704925, 0.4741772664955564, 0.0319540004265262, 4.0221212284407513e-07, 0.05827387838068315, 0.06401400633728511, 0.10533795250060955, 0.10846426726701881, 7.237620342665599e-09, 7.23762548297136e-09, 0.06828956430767279, 7.237682227000317e-09, 0.11054219089023525, 7.237920093341311e-09, 7.237810967882402e-09, 0.08790112550418577, 0.14758465847190738, 0.1734215383222743, 7.2378256021661625e-09, 0.08659395233578869, 0.9660918886619115, 7.237794944142397e-09, 0.055259723202446424, 7.895574123564978e-06, 0.2071598244973022, 0.2621295852512175, 7.237587218314025e-09, 7.23754515464561e-09, 7.237643899937859e-09, 7.237911933233457e-09, 1.6949050631640585, 7.2373451435798755e-09, 0.037322739329960906, 7.2370595752024335e-09, 7.237511666719978e-09, 0.0543861332327824, 7.236709485131116e-09, 0.1056581425387936, 7.236279967590209e-09, 7.237284513212124e-09, 0.11179704134255315, 7.237676291755024e-09, 0.02817428917594753, 0.0762465977153707, 7.236681734264762e-09, 0.053662441992030616, 7.238020438363776e-09, 1.9996231146566212, 0.032519579409532935, 0.25680691097042746, 7.237603555297153e-09, 7.2375600812115e-09, 7.237620315794036e-09, 0.08785284425351132, 0.216069555893414, 0.09325377220359012, 7.235952152134115e-09, 7.235396165804786e-09, 9.793665086575101e-07, 7.23631728256266e-09, 0.014340968396937834, 7.235872858059902e-09, 0.032613416615477224, 0.06026659619289539, 7.236884628108463e-09, 7.236308837320264e-09, 0.45524428781541343, 7.236605973239871e-09, 0.02232923746247239, 0.037544050418039536, 0.02760952589367134, 0.08143811963334276, 1.9996233204290161, 0.09373367357202689, 1.999623384128288, 0.0830993643128371, 0.03927605310177124, 7.237828635294698e-09, 7.237844619896721e-09, 0.1421035933345608, 0.08441849034745459, 2.9943575084011456e-06, 0.017582128752944043, 0.08881805441052429, 0.5148356147189795, 0.12111949696369854, 0.09282753170135083, 0.618215952237388, 7.2377889810485996e-09, 7.237796531228808e-09, 1.9996233660524054, 7.237840641589654e-09, 0.027912044814330354, 7.238007326489035e-09, 0.15165920290782692, 3.614899918219031e-06, 0.10488795960581691, 0.10356904897343606, 7.018147821478223e-06, 7.238150342653653e-09, 0.056818517566946475, 0.7806524360195004, 0.1041365872894135, 7.237878704489746e-09, 0.1503655911314277, 7.237586292696176e-09, 0.061362302464301736, 7.237981648558327e-09, 7.491395001816887e-06, 7.238146695472155e-09, 0.059245181979322256, 0.053020273324361177, 1.4564572770334996, 0.07512818836236977, 0.11967262761334616, 0.12448982006674117, 6.801601759285004e-06, 0.13132638311734177, 0.07440207024113797, 7.237937614046299e-09, 5.8970470493814676e-06, 0.09824157085943494, 0.2158044630070995, 7.237874513684678e-09, 0.3289703885526672, 7.238108047738471e-09, 0.1804099081849735, 0.09636594822503596, 5.159347271782781e-06, 0.33136725665388594, 0.00014237343667284212, 0.06879659092550618, 1.999623110873663, 0.14114802050410166, 7.083148264077046e-06, 7.238151659440171e-09, 3.052456198125419e-06, 7.237920101572182e-09, 7.236755945057567e-09, 0.03835795686683457, 7.237917690115282e-09, 0.07695565691071392, 4.755991990395707e-06, 0.07562498643431191, 1.9510129040973974e-07, 0.05509765915445351, 0.17701054500937274, 0.00011218575185868861, 7.2377108331571945e-09, 7.238727061244996e-06, 7.237815990923089e-09, 7.2378007761875665e-09, 7.237860769351968e-09, 0.0381380849049317, 7.2376957650742796e-09, 0.1437433240012085, 0.2733103925184933, 0.17149569647894958, 7.237670428543069e-09, 7.237701734970954e-09, 0.11048610788496865, 0.0710074824777896, 7.237962825086002e-09, 0.2544027525703811, 0.271223291423233, 0.21366875773039298, 0.07181517356237209, 0.00014224397935210003]
solution3 = [0.09845798601137727, 0.05235129051291274, 3.4637884724695923e-10, 1.2087245554283378e-10, 1.277243920458848e-10, 6.345043829229675e-11, 4.2603232361483807e-11, 0.04648143281760121, 0.10260869162545352, 1.3086889987777765e-10, 0.44315106943642774, 1.4881166729704977e-10, 0.4143557510974596, 0.4740387033878286, 0.03180629701517362, 6.60392786374749e-11, 0.05812691265728205, 0.06386898657537844, 0.10519545584815498, 0.10832024265077014, 5.028006518026124e-11, 1.595216277189811e-10, 0.06813675352472737, 7.814805003164243e-11, 0.11039847300615312, 9.31485793553421e-11, 1.1192816732728532e-10, 0.08775433996783769, 0.1474566966503736, 0.17328865665285167, 1.3212321337788378e-10, 0.0864447053328543, 0.9660184902470643, 5.529120907682417e-11, 0.05511595448009186, 3.702746397617944e-10, 0.20704611605238618, 0.26204030402150635, 7.212102760455414e-11, 1.937260553908852e-10, 7.916510144164216e-11, 8.684453972570563e-11, 1.6932868723137136, 1.8950956236518795e-10, 0.037177541764919685, 2.0145767231491818e-10, 1.6349510178353e-10, 0.054236332710144504, 2.0544368592438836e-10, 0.10551063508122654, 2.311663736481425e-10, 1.4219552648115858e-10, 0.11165632793882571, 1.7951674278996065e-10, 0.02802565727199464, 0.07609626464327118, 1.6398507600495067e-10, 0.05351662370708278, 1.1213139012720843e-10, 1.9999998409209256, 0.03237250887387575, 0.2566624822860445, 1.7678080459846958e-10, 1.2705492267816227e-10, 2.34457287498686e-10, 0.08770694636539007, 0.2159081791364836, 0.09310107903618703, 2.7721419921235305e-10, 2.786252386071498e-10, 8.513172677211416e-11, 2.1045107074832499e-10, 0.014189404203355604, 2.8349155779265384e-10, 0.032467845357393645, 0.06011532164732849, 1.3431835553242718e-10, 2.58854351885174e-10, 0.45511711482699563, 2.6588542505520313e-10, 0.022175146485019767, 0.03739372176281378, 0.027459070804997238, 0.08129443945083885, 1.9999999095367789, 0.09359211377263606, 1.9999999192413225, 0.08295965734521624, 0.03912869332721457, 1.0123661522978919e-10, 1.5176449708111553e-10, 0.1419558506740005, 0.08427488229948836, 1.2611663820690265e-10, 0.01742975451919606, 0.08867146989039837, 0.5147218925702534, 0.12097317228637067, 0.09268474681299321, 0.6180159856198104, 1.626127626916606e-10, 2.4324018919336226e-10, 1.999999924801794, 1.226927429791909e-10, 0.027760561007152667, 1.0242271520292864e-10, 0.15152989782008439, 1.9602529069763426e-10, 0.1047551266334289, 0.10343296976674446, 8.209111078353925e-10, 1.1884904323825162e-10, 0.056681520863819446, 0.7802662920219428, 0.10399368508038304, 1.379434171945706e-10, 0.15023123295080434, 2.0980222997127698e-10, 0.06121748604754761, 1.576384669048342e-10, 8.293448248484229e-10, 1.1436852596073048e-10, 0.05910015281679533, 0.052881364941078896, 1.456674374331224, 0.07497874937042981, 0.11952415418675523, 0.12435252124930413, 9.405748300063544e-10, 0.131188209541422, 0.07425468457905474, 1.3635309503266625e-10, 5.123476349077258e-10, 0.09809923404591936, 0.21563201708488866, 1.7241194872576032e-10, 0.32886510803690117, 2.0747322440570056e-10, 0.1802849839018422, 0.09622943292338379, 3.018496342032383e-10, 0.3312770377866408, 2.5592372730892823e-06, 0.0686541376295017, 1.9999999025282433, 0.1410064452606091, 7.419786634395349e-10, 1.3072670874311247e-10, 2.3804844058293656e-10, 1.8556600491771791e-10, 2.8027783678369253e-10, 0.03820879395863764, 1.9115259591977191e-10, 0.07681235704436908, 3.089637572888028e-10, 0.0754830605940923, 1.581041815325021e-10, 0.054954936254131004, 0.17685331275083357, 7.355142585462185e-08, 1.4339127114657462e-10, 1.0617703159326499e-09, 1.8992952457797828e-10, 1.4036683446215674e-10, 2.003478416866673e-10, 0.03798608218592753, 2.2015411725946376e-10, 0.14360563992313077, 0.2731960680288413, 0.17137641029663478, 1.722151074782117e-10, 1.3760732438833895e-10, 0.11033937488309772, 0.07086188641208419, 1.4771552456459296e-10, 0.25429479452157233, 0.27104379460191447, 0.21353862042342664, 0.0716725812647501, 1.611863407734671e-07]

In [ ]:
df = pd.read_csv("/kaggle/input/hull-tactical-market-prediction/train.csv").iloc[8990-180:8990] # for last analyzed public LB

df["prediction"] = solution1
score1 = get_score(df.copy(), df[["date_id", "prediction"]], "date_id")
print("Solution 1. Score:", score1)

df["prediction"] = solution2
score1 = get_score(df.copy(), df[["date_id", "prediction"]], "date_id")
print("Solution 2. Score:", score1)

df["prediction"] = solution3
score1 = get_score(df.copy(), df[["date_id", "prediction"]], "date_id")
print("Solution 3. Score:", score1)

Let's analyze distribution of $strategy\_returns$. All distributions are almost the same! 

In [ ]:
df["pred1"] = solution1
df["strategy_returns1"] = df['risk_free_rate'] + df['pred1'] * (df['forward_returns'] - df['risk_free_rate'])

df["pred2"] = solution2
df["strategy_returns2"] = df['risk_free_rate'] + df['pred2'] * (df['forward_returns'] - df['risk_free_rate'])

df["pred3"] = solution3
df["strategy_returns3"] = df['risk_free_rate'] + df['pred3'] * (df['forward_returns'] - df['risk_free_rate'])

fig, axes = plt.subplots(1, 3, figsize=(15, 5)) 

for i in range(3):
    ax = axes[i]
    ax.hist(x=df[f"strategy_returns{i+1}"], bins='auto', color='skyblue', edgecolor='black')
    mean1 = df[f"strategy_returns{i+1}"].mean()
    ax.axvline(x=mean1, color="red", linestyle="dashed", linewidth=2, label=f"Mean: {mean1:.9f}")
    ax.set_title(f"Strategy_returns {i+1} Distribution")
    ax.set_xlabel("Returns")
    ax.set_ylabel("Frequency")
    ax.legend()

Where $strategy\_returns$ is close to zero, prediction is also almost zero:

In [ ]:
print("Solution 1", (df[df["strategy_returns1"] < 0.0002]["pred1"] < 0.00001).mean()*100, "%")
print("Solution 2", (df[df["strategy_returns2"] < 0.0002]["pred2"] < 0.00001).mean()*100, "%")
print("Solution 3", (df[df["strategy_returns3"] < 0.0002]["pred3"] < 0.00001).mean()*100, "%")

Note, that $strategy\_returns1$ is close to zero only in cases where it is more profitable to invest in risk-free investments:

In [ ]:
for i in range(1, 4):
    cutted_df = df[df[f"strategy_returns{i}"] < 0.0002]
    print(f"Solution {i}", ((cutted_df["risk_free_rate"] - cutted_df["forward_returns"]) > 0).mean()*100, "%")

Where $strategy\_returns$ is not close to the biggest value, prediction is almost 2 (so it is implied then that it would be beneficial for us to get the global biggest value, but we could not achieve this because of MAX_INVESTMENT = 2):

In [ ]:
print("Solution 1", (df[(df["strategy_returns1"] > 0.0002) & (df["strategy_returns1"] < df["strategy_returns1"].max() - 1e-4)]["pred1"] > 1.999).mean()*100, "%")
print("Solution 2", (df[(df["strategy_returns2"] > 0.0002) & (df["strategy_returns2"] < df["strategy_returns2"].max() - 1e-4)]["pred2"] > 1.999).mean()*100, "%")
print("Solution 3", (df[(df["strategy_returns3"] > 0.0002) & (df["strategy_returns3"] < df["strategy_returns3"].max() - 1e-4)]["pred3"] > 1.999).mean()*100, "%")

So,
1) let's define $C$ as the optimal value of nonzero $strategy\_returns$
2) initialize it with the average value among the largest values of $strategy\_returns$ of all solutions
3) make a new prediction using $C$ (a known optimal $C$ allows us to get an optimal $prediction = \frac{strategy\_returns - risk\_free\_rate}{forward\_returns - risk\_free\_rate}$)
4) analyze the new distribution - it will be the same!

In [ ]:
C = (df["strategy_returns1"].max() + df["strategy_returns2"].max() + df["strategy_returns3"].max()) / 3
print("C:", C)
df["pred4"] = (C - df["risk_free_rate"]) / (df["forward_returns"] - df["risk_free_rate"])
df["pred4"] = df["pred4"].clip(0, 2)


# plotting
df["strategy_returns4"] = df['risk_free_rate'] + df['pred4'] * (df['forward_returns'] - df['risk_free_rate'])

plt.figure(figsize=(5, 5))
plt.hist(x=df[f"strategy_returns4"], bins='auto', color='skyblue', edgecolor='black')
mean1 = df[f"strategy_returns4"].mean()
plt.axvline(x=mean1, color="red", linestyle="dashed", linewidth=2, label=f"Mean: {mean1:.9f}")
plt.title(f"Custom strategy_returns Distribution")
plt.xlabel("Returns")
plt.ylabel("Frequency")
plt.legend()

## Solution: forecasting with known future

I propose a solution that looks for the optimal $C$, trying to maximize sharpe-metric

In [ ]:
df = pd.read_csv("/kaggle/input/hull-tactical-market-prediction/train.csv")
df = df.iloc[8990-180:8990] # for last analyzed public LB

In [ ]:
def get_public_score(C):
    df["prediction"] = (C - df["risk_free_rate"]) / (df["forward_returns"] - df["risk_free_rate"])
    df["prediction"] = df["prediction"].clip(0, 2)
    return get_score(df.copy(), df[["date_id", "prediction"]], "date_id")

Let's take a look at how $C$ and the sharpe are related. This chart shows a strong fluctuation. The overall trend looks like a parabola that is wobbling from top to bottom

In [ ]:
def visualize_interval(left, right, step, ax):
    """
    Visualizing interval [center-eps, center+eps) with step precision
    """
    scores = []
    for C in tqdm(np.arange(left, right, step)):
        scores.append(get_public_score(C))

    scores = np.array(scores)

    ax.plot(np.arange(left, right, step), scores, color='green', linewidth=1)
    ax.axhline(y=scores.max(), color='red', linestyle='-', linewidth=1, label=f"Maximum score: {scores.max()} with C={left+scores.argmax()*step}")
    
    ax.set_title(f"[{left};{right}) with step={step}")
    ax.set_xlabel("C")
    ax.set_ylabel("Scores")
    ax.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
    
    ax.legend()
    ax.grid(True, linestyle='--', alpha=0.6)

    return left+scores.argmax()*step

def visualize_interval_eps(center, eps, step, ax):
    """
    Visualizing interval [center-eps, center+eps) with step precision
    """
    return visualize_interval(max(0, center-eps), center+eps, step, ax)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(20, 10))
plt.title('C-to-Score relation')
visualize_interval(0, 1, 1e-2, axes[0][0])
visualize_interval(0, 0.005, 1e-5, axes[0][1])
visualize_interval(0.00074, 0.000741, 1e-9, axes[1][0])
visualize_interval(0.000740420, 0.000740421, 1e-12, axes[1][1])
plt.show()

Let's maximize public score using $C$

In [ ]:
def standard_optimize(f, x0, bounds, maxiter, tolerances):
    """
    Try all methods of optimizations
    """
    methods = [
        'Nelder-Mead', 'Powell', 'CG', 'BFGS', 'TNC', 'COBYLA', 'SLSQP'
    ]

    bestresult = {'score': np.inf}

    for method in methods:
        for tol in tolerances:
            options = {'disp': False, 'tol': tol}
            if maxiter is not None:
                options['maxiter'] = maxiter
            res = minimize(
                f,
                x0,
                method=method,
                bounds=bounds,
                options=options
            )

            score = res.fun
            C = res.x[0]

            if score < bestresult['score']:
                bestresult = {
                    'C': C,
                    'score': score,
                    'method': method,
                    'tolerance': tol,
                    'success': res.success,
                    'message': res.message,
                    'nfev': res.nfev
                }

    return bestresult

In [ ]:
def custom_minimize(f, x0, eps0, step0, k, depth, cur_eps_decrease_coef=10, cur_step_decrease_coef=10):
    """This function performs iterative optimization:

    1. explores the epsilon neighborhood of the k current best candidate points
    2. selects the k best points from intervals [candidate-eps; candidate+eps) with step cur_step
    3. with each iteration, the search range (cur_eps) and step (cur_step) decrease, gradually narrowing the search area and clarifying the minimum.
    """
    topk_minimums = [(f(x0), x0)]
    cur_eps = eps0
    cur_step = step0
    for cur_depth in range(depth):
        scores = topk_minimums.copy()
        for _, candidate in topk_minimums:
            for x in tqdm(np.arange(candidate - cur_eps, candidate + cur_eps, cur_step)):
                scores.append((f(x), x))
        scores.sort()
        topk_minimums = scores[:k]

        cur_eps /= cur_eps_decrease_coef
        cur_step /= cur_step_decrease_coef
        
        print(f"Depth {cur_depth}. Top minimum: {topk_minimums}")

    return min(topk_minimums)

In [ ]:
C1_data = standard_optimize(lambda x: -get_public_score(x), x0=0.00074042005, bounds=Bounds(lb=0.000740420, ub=0.000740421), maxiter=None, tolerances=[10 ** (-i) for i in range(21)])
print(C1_data)

In [ ]:
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    C2_data = (-17.396313685750652, 0.0007404204623568324)
else:
    df = pd.read_csv("/kaggle/input/hull-tactical-market-prediction/train.csv")
    df = df.iloc[8990-180:8990] # for all on last seend public
    C2_data = custom_minimize(lambda x: -get_public_score(x), x0=0.0007404204623568324, eps0=1e-10, step0=1e-15, k=1, depth=5)
print(C2_data)

Next, let's take the most optimal $C$ among all methods (it is optimal with my custom optimization algorithm)

## Trivial submission with precalculated C

In [ ]:
C = C2_data[1]

train = pd.read_csv(
    f"/kaggle/input/hull-tactical-market-prediction/train.csv", 
    index_col = "date_id"
).iloc[8990-180:8990].reset_index() # for all on last seend public
train["prediction"] = (C - train["risk_free_rate"]) / (train["forward_returns"] - train["risk_free_rate"])
train["prediction"] = train["prediction"].clip(0, 2)
ans = list(train["prediction"])

testrow = 0

def predict(test) -> float:
    global testrow
    testrow += 1
    return float(ans[testrow - 1])

In [ ]:
inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(('/kaggle/input/hull-tactical-market-prediction/',))